# Predicting the season
This is where things get extremely arbitrary. Each team will be given a rating purely based off my intuition as a basketball fan. Each team will be given an advantage or disadvantage in each game depending on whether they're home or away or based on how many days rest they have had before the game. There is also an element of randomness in basketball. When predicting each game I will include a random variable that will accountable for 15% of the likelihood of the outcome of the game.

## Imports

In [1]:
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt

import time

import datetime

import random

# Loading the NBA Games

In [2]:
df = pd.read_excel('../NBA_game_data.xlsx')
df.shape, df.dtypes

((1230, 67),
 Date                    datetime64[ns]
 Away_team                       object
 Home_team                       object
 Philadelphia 76ers               int64
 Los Angeles Lakers               int64
                              ...      
 Miami Heat last game    datetime64[ns]
 away_last_date          datetime64[ns]
 home_last_Date          datetime64[ns]
 away_team_dslg                   int64
 home_team_dslg                   int64
 Length: 67, dtype: object)

# Rating each team
Ratings will be floats between 0.0 and 10.0

In [3]:
df['Away_team'].unique()

array(['Philadelphia 76ers', 'Los Angeles Lakers', 'Orlando Magic',
       'Washington Wizards', 'Houston Rockets', 'New Orleans Pelicans',
       'New York Knicks', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Oklahoma City Thunder', 'Charlotte Hornets', 'Denver Nuggets',
       'Dallas Mavericks', 'Portland Trail Blazers', 'Milwaukee Bucks',
       'Los Angeles Clippers', 'San Antonio Spurs', 'Toronto Raptors',
       'Boston Celtics', 'Detroit Pistons', 'Memphis Grizzlies',
       'Utah Jazz', 'Phoenix Suns', 'Minnesota Timberwolves',
       'Sacramento Kings', 'Indiana Pacers', 'Brooklyn Nets',
       'Golden State Warriors', 'Atlanta Hawks', 'Miami Heat'],
      dtype=object)

In [4]:
team_ratings = {'Philadelphia 76ers':8.8, 'Los Angeles Lakers':7.9, 'Orlando Magic':7.0,
       'Washington Wizards':7.4, 'Houston Rockets':6.0, 'New Orleans Pelicans':8.4,
       'New York Knicks':7.4, 'Chicago Bulls':7.9, 'Cleveland Cavaliers':8.4,
       'Oklahoma City Thunder':6.5, 'Charlotte Hornets':6.7, 'Denver Nuggets':9.1,
       'Dallas Mavericks':8.7, 'Portland Trail Blazers':8.0, 'Milwaukee Bucks':9.0,
       'Los Angeles Clippers':9.0, 'San Antonio Spurs':6.0, 'Toronto Raptors':8.2,
       'Boston Celtics':8.8, 'Detroit Pistons':6.8, 'Memphis Grizzlies':8.4,
       'Utah Jazz':6.0, 'Phoenix Suns':8.6, 'Minnesota Timberwolves':8.6,
       'Sacramento Kings':7.7, 'Indiana Pacers':6.6, 'Brooklyn Nets':8.7,
       'Golden State Warriors':8.9, 'Atlanta Hawks':8.4, 'Miami Heat':8.4}

In [5]:
len(team_ratings)

30

# Add team ratings to Dataframe

In [6]:
df['Away_team_rating'] = df['Away_team'].map(team_ratings)
df['Home_team_rating'] = df['Home_team'].map(team_ratings)

In [7]:
df.head(2)

,Date,Away_team,Home_team,Philadelphia 76ers,Los Angeles Lakers,Orlando Magic,Washington Wizards,Houston Rockets,New Orleans Pelicans,New York Knicks,...,Brooklyn Nets last game,Golden State Warriors last game,Atlanta Hawks last game,Miami Heat last game,away_last_date,home_last_Date,away_team_dslg,home_team_dslg,Away_team_rating,Home_team_rating
0,2022-10-18,Philadelphia 76ers,Boston Celtics,1,0,0,0,0,0,0,...,NaT,NaT,NaT,NaT,2022-04-09,2022-04-09,192,192,8.8,8.8
1,2022-10-18,Los Angeles Lakers,Golden State Warriors,0,1,0,0,0,0,0,...,NaT,2022-04-09,NaT,NaT,2022-04-09,2022-04-09,192,192,7.9,8.9


# Simulating games

In [8]:
team_wins = {'Philadelphia 76ers':0, 'Los Angeles Lakers':0, 'Orlando Magic':0,
       'Washington Wizards':0, 'Houston Rockets':0, 'New Orleans Pelicans':0,
       'New York Knicks':0, 'Chicago Bulls':0, 'Cleveland Cavaliers':0,
       'Oklahoma City Thunder':0, 'Charlotte Hornets':0, 'Denver Nuggets':0,
       'Dallas Mavericks':0, 'Portland Trail Blazers':0, 'Milwaukee Bucks':0,
       'Los Angeles Clippers':0, 'San Antonio Spurs':0, 'Toronto Raptors':0,
       'Boston Celtics':0, 'Detroit Pistons':0, 'Memphis Grizzlies':0,
       'Utah Jazz':0, 'Phoenix Suns':0, 'Minnesota Timberwolves':0,
       'Sacramento Kings':0, 'Indiana Pacers':0, 'Brooklyn Nets':0,
       'Golden State Warriors':0, 'Atlanta Hawks':0, 'Miami Heat':0}

Loop through dataframe

In [9]:
teams = df['Away_team'].unique()
wins_df = pd.DataFrame(teams)
for i in range(1000):
    team_wins_copy = team_wins.copy()
    for index, row in df.iterrows():
        away_team_rating = row['Away_team_rating'] * 0.97 # slight disadvantage for playing away
        home_team_rating = row['Home_team_rating']

        # penalise away team for limited rest days
        if row['away_team_dslg'] == 1:
            away_team_rating = away_team_rating * 0.85
        elif row['away_team_dslg'] == 2:
            away_team_rating = away_team_rating * 0.9
        elif row['away_team_dslg'] == 3:
            away_team_rating = away_team_rating * 0.95

        # penalise home team for limited rest days   
        if row['home_team_dslg'] == 1:
            home_team_rating = home_team_rating * 0.85
        elif row['home_team_dslg'] == 2:
            home_team_rating = home_team_rating * 0.9
        elif row['home_team_dslg'] == 3:
            home_team_rating = home_team_rating * 0.95

        away_team_rating = (away_team_rating * 0.75) + (random.randint(0, 10) * 0.25)
        home_team_rating = (home_team_rating * 0.75) + (random.randint(0, 10) * 0.25)

        if away_team_rating > home_team_rating:
            team_wins_copy[row['Away_team']] = team_wins_copy[row['Away_team']] + 1
        elif home_team_rating > away_team_rating:
            team_wins_copy[row['Home_team']] = team_wins_copy[row['Home_team']] + 1
        elif home_team_rating == away_team_rating:
            # if the ratings are tied then the home team wins
            team_wins_copy[row['Home_team']] = team_wins_copy[row['Home_team']] + 1

        wins_df[i+1] = wins_df[0].map(team_wins_copy)

In [10]:
wins_df

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,1000
0,Philadelphia 76ers,51,53,60,48,53,49,61,57,56,...,57,47,55,50,56,58,59,44,55,53
1,Los Angeles Lakers,39,40,38,43,46,45,35,39,39,...,42,34,43,38,37,43,41,46,48,45
2,Orlando Magic,32,27,26,23,27,24,29,28,23,...,30,21,27,24,28,24,27,31,26,26
3,Washington Wizards,33,28,31,30,26,31,32,36,34,...,27,36,35,36,31,33,34,31,34,31
4,Houston Rockets,22,16,12,15,9,17,20,9,14,...,24,19,17,15,13,16,17,12,11,8
5,New Orleans Pelicans,49,49,50,47,54,48,52,50,45,...,44,47,49,39,48,49,52,50,46,47
6,New York Knicks,28,40,37,36,33,32,33,38,32,...,32,31,34,30,36,34,31,40,27,28
7,Chicago Bulls,38,41,50,38,34,36,37,38,38,...,40,43,32,39,45,38,44,41,35,40
8,Cleveland Cavaliers,55,51,49,51,56,60,53,44,50,...,52,46,48,49,45,53,56,45,53,51
9,Oklahoma City Thunder,23,13,19,20,14,18,21,17,21,...,21,13,21,20,19,18,22,21,23,24


# Mean wins

In [11]:
wins_df['mean_wins'] = wins_df.iloc[:, 1:1001].mean(axis=1).round()

In [12]:
wins_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,992,993,994,995,996,997,998,999,1000,mean_wins
0,Philadelphia 76ers,51,53,60,48,53,49,61,57,56,...,47,55,50,56,58,59,44,55,53,55.0
1,Los Angeles Lakers,39,40,38,43,46,45,35,39,39,...,34,43,38,37,43,41,46,48,45,41.0


# Split by Conference

In [13]:
wins_df[0].unique()

array(['Philadelphia 76ers', 'Los Angeles Lakers', 'Orlando Magic',
       'Washington Wizards', 'Houston Rockets', 'New Orleans Pelicans',
       'New York Knicks', 'Chicago Bulls', 'Cleveland Cavaliers',
       'Oklahoma City Thunder', 'Charlotte Hornets', 'Denver Nuggets',
       'Dallas Mavericks', 'Portland Trail Blazers', 'Milwaukee Bucks',
       'Los Angeles Clippers', 'San Antonio Spurs', 'Toronto Raptors',
       'Boston Celtics', 'Detroit Pistons', 'Memphis Grizzlies',
       'Utah Jazz', 'Phoenix Suns', 'Minnesota Timberwolves',
       'Sacramento Kings', 'Indiana Pacers', 'Brooklyn Nets',
       'Golden State Warriors', 'Atlanta Hawks', 'Miami Heat'],
      dtype=object)

In [14]:
west = ['Los Angeles Lakers', 'Houston Rockets', 'New Orleans Pelicans',
       'Oklahoma City Thunder', 'Denver Nuggets', 'Dallas Mavericks',
        'Portland Trail Blazers', 'Los Angeles Clippers', 'San Antonio Spurs',
       'Memphis Grizzlies', 'Utah Jazz', 'Phoenix Suns', 'Minnesota Timberwolves',
       'Sacramento Kings', 'Golden State Warriors']

east = ['Philadelphia 76ers', 'Orlando Magic', 'Washington Wizards',
        'New York Knicks', 'Chicago Bulls', 'Cleveland Cavaliers',
        'Charlotte Hornets', 'Milwaukee Bucks', 'Toronto Raptors',
       'Boston Celtics', 'Detroit Pistons', 'Indiana Pacers', 'Brooklyn Nets',
       'Atlanta Hawks', 'Miami Heat']

In [32]:
west_df = pd.DataFrame(west)
east_df = pd.DataFrame(east)

In [33]:
team_win_dict = wins_df[[0, 'mean_wins']].set_index(0).to_dict('dict')['mean_wins']

In [34]:
west_df['Wins'] = west_df[0].map(team_win_dict).astype(int)
east_df['Wins'] = east_df[0].map(team_win_dict).astype(int)

In [35]:
west_df = west_df.rename(columns={0:"Team"})
east_df = east_df.rename(columns={0:"Team"})

In [36]:
west_df['Losses'] = 82 - west_df['Wins']
east_df['Losses'] = 82 - east_df['Wins']

In [44]:
from IPython.display import HTML

In [50]:
HTML(west_df.sort_values(by=['Wins'], ascending=False).reset_index(drop=True).set_index([pd.Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])]).to_html())

,Team,Wins,Losses
1,Denver Nuggets,59,23
2,Los Angeles Clippers,59,23
3,Golden State Warriors,56,26
4,Dallas Mavericks,55,27
5,Minnesota Timberwolves,54,28
6,Phoenix Suns,52,30
7,Memphis Grizzlies,49,33
8,New Orleans Pelicans,48,34
9,Portland Trail Blazers,43,39
10,Los Angeles Lakers,41,41


In [52]:
HTML(east_df.sort_values(by=['Wins'], ascending=False).reset_index(drop=True).set_index([pd.Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15])]).to_html())

,Team,Wins,Losses
1,Milwaukee Bucks,58,24
2,Philadelphia 76ers,55,27
3,Boston Celtics,55,27
4,Brooklyn Nets,53,29
5,Cleveland Cavaliers,50,32
6,Miami Heat,50,32
7,Atlanta Hawks,49,33
8,Toronto Raptors,46,36
9,Chicago Bulls,40,42
10,Washington Wizards,33,49
